In [1]:
import requests as req
import json
from citipy import citipy # downloaded from https://pypi.python.org/pypi/citipy
# https://github.com/wingchen/citipy for how to use it and more info about it

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# coordinates: (latitude (horizontal), longitude (vertical))
    #coordinate range: ([-90,90], [-180, 180])
city = citipy.nearest_city(-100, 90).city_name
#city.city_name

city

'albany'

In [70]:
# url, api key for openweathermap (OWM)
url = ' http://api.openweathermap.org/data/2.5/weather' #For how to use OWM's api: https://openweathermap.org/current
apiKey = 'c8e2baf63c2571f94bf9d95c94483400' # my requested api key

payload_Test = {'APPID':apiKey, 'units':'imperial', 'q': 'london'} # q doesn't seem to be case sensitive


testResp = req.get(url, params=payload_Test)
weather = testResp.json()

weather

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 51.51, 'lon': -0.13},
 'dt': 1512454800,
 'id': 2643743,
 'main': {'humidity': 81,
  'pressure': 1036,
  'temp': 44.4,
  'temp_max': 46.4,
  'temp_min': 42.8},
 'name': 'London',
 'sys': {'country': 'GB',
  'id': 5093,
  'message': 0.1708,
  'sunrise': 1512460180,
  'sunset': 1512489164,
  'type': 1},
 'visibility': 10000,
 'weather': [{'description': 'overcast clouds',
   'icon': '04n',
   'id': 804,
   'main': 'Clouds'}],
 'wind': {'deg': 270, 'speed': 6.93}}

In [72]:
# Sample of 500 countries from citipy
sample_ls = []
Lati_ls = []
Longi_ls = []

#loop to fill the sample list
#TEST for 10 samples
while len(sample_ls) < 10:
#TESTING USING A FOR LOOP: for i in range(0, 10):
    Lati = np.random.uniform(low=-90, high=90)
    Longi = np.random.uniform(low=-180, high=180)

    #get nearest city name
    city = citipy.nearest_city(Lati, Longi).city_name
    #check if city name is in the sample list
    duplicate = False
    for entry in sample_ls:
        if city == entry:
            duplicate = True
            break
        if city != entry:
            continue
    if duplicate == False:
        #storing Latitude and Longitude to make sure range of each is OK
        Lati_ls.append(Lati)
        Longi_ls.append(Longi)
        sample_ls.append(city)

columns=['Country', 'Date', 'Cloudiness', 'Humidity', 'Latitude', 'Longitude', 'Max Temp', 'Wind Speed (mph)']
        
citySample_df = pd.DataFrame(index=sample_ls, columns=columns)

citySample_df
    

,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed (mph)
arraial do cabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
norman wells,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pindiga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
puerto ayora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alofi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mundi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longyearbyen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
busselton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
santa luzia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chake chake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# url, api key for openweathermap (OWM)
url = ' http://api.openweathermap.org/data/2.5/weather' #For how to use OWM's api: https://openweathermap.org/current
apiKey = 'c8e2baf63c2571f94bf9d95c94483400' # my requested api key

# loop through dataframe to get values for each index
count = 0
for index, row in citySample_df.iterrows():
    payload_sampl = {'APPID':apiKey, 'units':'imperial', 'q': index} # q doesn't seem to be case sensitive
    weather = req.get(url, params=payload_sampl).json()
    try:     
        if weather['cod'] != '404': #'cod' seems to equal '404' if city isn't found in OWM. 'cod' == 200 otherwise 
            row['Country'] = weather['sys']['country']
            row['Date'] = weather['dt']
            row['Latitude'] = weather['coord']['lat']
            row['Longitude'] = weather['coord']['lon']
            row['Max Temp'] = weather['main']['temp_max']
            row['Humidity'] = weather['main']['humidity']
            row['Cloudiness'] = weather['clouds']['all'] #find measure of cloudiness
            row['Wind Speed (mph)'] = weather['wind']['speed']
        else:
            citySample_df.drop(index, inplace=True)
    except ValueError:
        citySample_df.drop(index, inplace=True)
        continue
        
    count += 1

citySample_df

,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed (mph)
arraial do cabo,BR,1512456760,88,99,-22.97,-42.03,73.22,12.24
norman wells,CA,1512455100,75,84,65.28,-126.83,5,3.36
pindiga,NG,1512456761,68,39,9.98,10.93,63.23,5.41
puerto ayora,EC,1512456761,56,99,-0.74,-90.35,70.43,9.1
alofi,NU,1512453600,88,88,-19.06,-169.92,80.6,5.82
mundi,IN,1512456762,100,55,22.07,76.5,71.42,7.54
longyearbyen,SJ,1512453000,75,62,78.22,15.64,23,12.75
busselton,AU,1512456762,0,97,-33.65,115.33,69.35,18.05
santa luzia,BR,1512453600,75,88,-19.77,-43.85,71.6,2.73
chake chake,TZ,1512456763,32,91,-5.25,39.77,83.52,4.74


In [78]:
#drop any NaN just in case
citySample_df.dropna(axis=0, how='any', inplace=True)

#format date from unix timestamp 
citySample_df['Date'] = pd.to_datetime(citySample_df['Date'], unit='s')

citySample_df

,Country,Date,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed (mph)
arraial do cabo,BR,2017-12-05 06:52:40,88,99,-22.97,-42.03,73.22,12.24
norman wells,CA,2017-12-05 06:25:00,75,84,65.28,-126.83,5,3.36
pindiga,NG,2017-12-05 06:52:41,68,39,9.98,10.93,63.23,5.41
puerto ayora,EC,2017-12-05 06:52:41,56,99,-0.74,-90.35,70.43,9.1
alofi,NU,2017-12-05 06:00:00,88,88,-19.06,-169.92,80.6,5.82
mundi,IN,2017-12-05 06:52:42,100,55,22.07,76.5,71.42,7.54
longyearbyen,SJ,2017-12-05 05:50:00,75,62,78.22,15.64,23,12.75
busselton,AU,2017-12-05 06:52:42,0,97,-33.65,115.33,69.35,18.05
santa luzia,BR,2017-12-05 06:00:00,75,88,-19.77,-43.85,71.6,2.73
chake chake,TZ,2017-12-05 06:52:43,32,91,-5.25,39.77,83.52,4.74


In [ ]:
#Scatterplot of Temperature (F) vs. Latitude
